In [1]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer


Using TensorFlow backend.
C:\Users\Ishant.Somal\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ishant.Somal\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ishant.Somal\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ishant.Somal\Anaconda3\lib\site-packages\tensorflo

In [2]:
# Define a flask app
app = Flask(__name__)

In [3]:
from keras.applications.resnet50 import ResNet50
model = ResNet50(weights='imagenet')

In [4]:
model

In [5]:
model.save(r'C:\Users\Ishant.Somal\OneDrive - EY\Desktop\Deployment-Deep-Learning-Model-master\Deployment-Deep-Learning-Model-master\models/model_resnet.h5')

OSError: Unable to create file (unable to open file: name = 'C:\Users\Ishant.Somal\OneDrive - EY\Desktop\Deployment-Deep-Learning-Model-master\Deployment-Deep-Learning-Model-master\models/model_resnet.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [ ]:
MODEL_PATH = r'C:\Users\Ishant.Somal\OneDrive - EY\Desktop\Deployment-Deep-Learning-Model-master\Deployment-Deep-Learning-Model-master\models/model_resnet.h5'

In [ ]:
model = load_model(MODEL_PATH)
model._make_predict_function()

In [ ]:
def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    # Be careful how your trained model deals with the input
    # otherwise, it won't make correct prediction!
    x = preprocess_input(x, mode='caffe')

    preds = model.predict(x)
    return preds

In [ ]:
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

In [ ]:
@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(
            basepath, r'C:\Users\Ishant.Somal\OneDrive - EY\Desktop\Deployment-Deep-Learning-Model-master\Deployment-Deep-Learning-Model-master\uploads', secure_filename(f.filename))
        f.save(file_path)
        # Make prediction
        preds = model_predict(r'C:\Users\Ishant.Somal\OneDrive - EY\Desktop\Deployment-Deep-Learning-Model-master\Deployment-Deep-Learning-Model-master\uploads', model)

        # Process your result for human
        # pred_class = preds.argmax(axis=-1)            # Simple argmax
        pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
        result = str(pred_class[0][0][1])               # Convert to string
        return result
    return None

       

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)